In [1]:
!git clone https://github.com/zzh8829/yolov3-tf2
%cd yolov3-tf2/
!pip install -r requirements-gpu.txt

Cloning into 'yolov3-tf2'...
remote: Enumerating objects: 419, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 419 (delta 2), reused 0 (delta 0), pack-reused 412
Receiving objects: 100% (419/419), 4.24 MiB | 7.07 MiB/s, done.
Resolving deltas: 100% (239/239), done.
/content/yolov3-tf2
Obtaining file:///content/yolov3-tf2 (from -r requirements-gpu.txt (line 6))
     |████████████████████████████████| 394.5 MB 35 kB/s 
     |████████████████████████████████| 28.7 MB 53 kB/s 
     |████████████████████████████████| 3.8 MB 53.0 MB/s 
     |████████████████████████████████| 2.9 MB 36.6 MB/s 
     |████████████████████████████████| 462 kB 36.5 MB/s 
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.34.1
    Uninstalling grpcio-1.34.1:
      Successfully uninstalled grpcio-1.34.1
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.5.0
    Uninstalling tensorfl

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd /content/yolov3-tf2/
!ls

import tensorflow as tf
tf.__version__

/content/yolov3-tf2
checkpoints	 data		  README.md		train.py
colab_gpu.ipynb  detect.py	  requirements-gpu.txt	yolov3_tf2
conda-cpu.yml	 detect_video.py  requirements.txt	yolov3_tf2.egg-info
conda-gpu.yml	 docs		  setup.py
convert.py	 LICENSE	  tools


'2.4.2'

##Convert Pretrained Darknet Weight

In [4]:
# %cd 


In [5]:
!wget https://pjreddie.com/media/files/yolov3.weights


--2021-07-31 05:41:03--  https://pjreddie.com/media/files/yolov3.weights
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 248007048 (237M) [application/octet-stream]
Saving to: ‘yolov3.weights’

yolov3.weights      100%[===================>] 236.52M  74.2MB/s    in 3.4s    

2021-07-31 05:41:07 (69.6 MB/s) - ‘yolov3.weights’ saved [248007048/248007048]



In [6]:
# --tiny yolov3-tiny --num_classes 1
!python convert.py --weights /content/drive/MyDrive/UAV/Model/data/yolov3.weights --output /content/drive/MyDrive/UAV/Model/checkpoints/yolov3.tf

2021-07-31 05:41:58.852337: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2021-07-31 05:42:00.394836: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-07-31 05:42:00.395749: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-07-31 05:42:00.453329: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-07-31 05:42:00.453919: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla T4 computeCapability: 7.5
coreClock: 1.59GHz coreCount: 40 deviceMemorySize: 14.75GiB deviceMemoryBandwidth: 298.08GiB/s
2021-07-31 05:42:00.453954: I tensorflow/stream_executor/platform/default/dso_loade

##Transfer Learning

In [7]:
pathDataset = "/content/drive/MyDrive/UAV/Model/train_image_folder"
output_path = "/content/drive/MyDrive/UAV/Model/data"
pathCSV = f"{output_path}/images_info.csv"

pathResizeImages = f"{output_path}/images_resize"
pathResizeCSV = f"{output_path}/images_info_resize.csv"


imageTest = "/content/drive/MyDrive/UAV/Model/train_image_folder/000.jpg"
annoYolo = "/content/drive/MyDrive/UAV/Model/yolo_anno"
annoVoc = "/content/drive/MyDrive/UAV/Model/voc_anno"

filenameTrain = f"{output_path}/train_labels.csv"
filenameTest = f"{output_path}/test_labels.csv"

In [9]:
%%bash
#!/bin/bash

touch /content/drive/MyDrive/UAV/Model/data/uav.names

__usage="person"


echo "$__usage" > /content/drive/MyDrive/UAV/Model/data/uav.names
cd data
cat /content/drive/MyDrive/UAV/Model/data/uav.names

person


In [10]:
!mkdir /content/drive/MyDrive/UAV/Model/checkpoints
# file already exists

mkdir: cannot create directory ‘/content/drive/MyDrive/UAV/Model/checkpoints’: File exists


!python /content/drive/MyDrive/UAV/Model/create_tfrecord.py 

--csv_input=/content/drive/MyDrive/UAV/Model/data/train_labels.csv  

--output_path=/content/drive/MyDrive/UAV/Model/data/train.record

--image_dir=/content/drive/MyDrive/UAV/Model/data/images_resize

In [11]:
!python train.py \
--dataset /content/drive/MyDrive/UAV/Model/data/train.record \
	--val_dataset /content/drive/MyDrive/UAV/Model/data/test.record \
	--classes /content/drive/MyDrive/UAV/Model/data/uav.names \
	--num_classes 1 \
	--mode fit --transfer darknet \
	--batch_size 8 \
	--epochs 100 \
	--weights /content/drive/MyDrive/UAV/Model/checkpoints/yolov3.tf \
	--weights_num_classes 80 \
  --size 320


2021-07-31 05:45:39.486401: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2021-07-31 05:45:41.211678: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-07-31 05:45:41.214089: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-07-31 05:45:41.248340: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-07-31 05:45:41.248952: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla T4 computeCapability: 7.5
coreClock: 1.59GHz coreCount: 40 deviceMemorySize: 14.75GiB deviceMemoryBandwidth: 298.08GiB/s
2021-07-31 05:45:41.248986: I tensorflow/stream_executor/platform/default/dso_loade

In [28]:
# !python train.py \
# --dataset /content/drive/MyDrive/UAV/Model/data/train.record \
# 	--val_dataset /content/drive/MyDrive/UAV/Model/data/test.record \
# 	--classes /content/drive/MyDrive/UAV/Model/data/uav.names \
# 	--num_classes 1 \
# 	--mode fit --transfer darknet \
# 	--batch_size 8 \
# 	--epochs 100 \
# 	--weights /content/drive/MyDrive/UAV/Model/checkpoints/yolov3.tf \
# 	--weights_num_classes 80 \
#   --size 320
!ls

import sys
from absl import app, logging, flags
from absl.flags import FLAGS
import time
import cv2
import numpy as np
import tensorflow as tf
from yolov3_tf2.models import (
    YoloV3, YoloV3Tiny
)
from yolov3_tf2.dataset import transform_images, load_tfrecord_dataset
from yolov3_tf2.utils import draw_outputs

def del_all_flags(FLAGS):
    flags_dict = FLAGS._flags()
    keys_list = [keys for keys in flags_dict]
    
    for keys in keys_list:
    	FLAGS.__delattr__(keys)
        
del_all_flags(FLAGS)

flags.DEFINE_string('classes', './data/coco.names', 'path to classes file')
flags.DEFINE_string('weights', './checkpoints/yolov3.tf',
                    'path to weights file')
flags.DEFINE_boolean('tiny', False, 'yolov3 or yolov3-tiny')
flags.DEFINE_integer('size', 416, 'resize images to')
flags.DEFINE_string('image', './data/girl.png', 'path to input image')
flags.DEFINE_string('tfrecord', None, 'tfrecord instead of image')
flags.DEFINE_string('output', './output.jpg', 'path to output image')
flags.DEFINE_integer('num_classes', 80, 'number of classes in the model')

app._run_init(['yolov3'], app.parse_flags_with_usage)

physical_devices = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)


FLAGS.classes = '/content/drive/MyDrive/UAV/Model/data/uav.names'
FLAGS.weights = '/content/drive/MyDrive/UAV/Model/checkpoints/yolov3.tf'
FLAGS.image = '/content/drive/MyDrive/UAV/Model/data/images_resize/000.jpg'
FLAGS.num_classes = 1
FLAGS.size = 320

# Lower threshold due to insufficient training
FLAGS.yolo_iou_threshold = 0.2
FLAGS.yolo_score_threshold = 0.2

if FLAGS.tiny:
    yolo = YoloV3Tiny(classes=FLAGS.num_classes)
else:
    yolo = YoloV3(classes=FLAGS.num_classes)

yolo.load_weights(FLAGS.weights).expect_partial()
logging.info('weights loaded')

class_names = [c.strip() for c in open(FLAGS.classes).readlines()]
logging.info('classes loaded')

img_raw = tf.image.decode_image(
    open(FLAGS.image, 'rb').read(), channels=3)

img = tf.expand_dims(img_raw, 0)
img = transform_images(img, FLAGS.size)

t1 = time.time()
boxes, scores, classes, nums = yolo(img)
t2 = time.time()
logging.info('time: {}'.format(t2 - t1))

logging.info('detections:')
for i in range(nums[0]):
    logging.info('\t{}, {}, {}'.format(class_names[int(classes[0][i])],
                                        np.array(scores[0][i]),
                                        np.array(boxes[0][i])))

img = cv2.cvtColor(img_raw.numpy(), cv2.COLOR_RGB2BGR)
img = draw_outputs(img, (boxes, scores, classes, nums), class_names)

from IPython.display import Image, display
display(Image(data=bytes(cv2.imencode('.jpg', img)[1]), width=800))

checkpoints	 data		  logs			tools
colab_gpu.ipynb  detect.py	  README.md		train.py
conda-cpu.yml	 detect_video.py  requirements-gpu.txt	yolov3_tf2
conda-gpu.yml	 docs		  requirements.txt	yolov3_tf2.egg-info
convert.py	 LICENSE	  setup.py		yolov3.weights


UnrecognizedFlagError: ignored

1. replace flag
2. path problem (path of saving model
3. modify the source code with output